## **Setup**

In [ ]:
import os
import cv2
from skimage.io import imread
from skimage.transform import resize
from skimage.color import rgb2gray
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## **Preparing Data**

In [ ]:
data_dir = '/content/gdrive/MyDrive/CCI/TCC/BaseImagensTecido/FS_1'

The function `listdir()` returns a list containing all directories within `data_dir` which will be used as the classes (`categories`) of the model.
The word `classes` would be more adequate, but `class` is a reserved word so then using `categories` prevents confusion.

In [ ]:
categories = os.listdir(data_dir) # Returns the classes of the model
print(categories)

['RC', 'LV', 'LH', 'OK']


The following snippet presents a way to load the custmized dataset (`dataset loading`) from an specific directory. Libraries like `TensorFlow`, `PyTorch`, `Fast.ai` e `YOLO` have already methods to load a dataset, however, the strategy is very similar. The function `load_image_dataset()` was created so that this chunck of code could be reutilized whenever needed.

In [ ]:
def load_image_dataset(directory, dimensions):

  '''
  This function receives a directory composed of directories which
  contain de image data. Each inside directory stand for a classification
  category. The function returns two arrays: data and labels, already
  transformed into numpy array object-type.

  The labels are identified by integer numbers following Python native
  indexation, however, it can converted to a string class.

  Two argument are required:
      directory, dimensions

  The dimensions will be used for resizing the images. After that, the flatten()
  method is used to convert images into arrays, i.e., from 2D objects into a
  single array (1D) object.

  By using the command `cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)` the images is
  converted from 3 chanels to 1 chanel (gray). This operation is important so
  the data is generated correctly.

  Uncomment the first lines of the code if you want to reuse this function in
  another script or notebook.
  '''

  #import os
  #from skimage.io import imread
  #from skimage.transform import resize
  #import numpy as np

  data = []
  labels = []

  for category_idx, category in enumerate(categories):
      for file in os.listdir(os.path.join(data_dir, category)):
          img_path = os.path.join(data_dir, category, file)
          img = cv2.imread(img_path)
          img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
          img = cv2.resize(img, (128, 128))
          img = img / 255
          img = img.flatten()
          data.append(img) # img.flatten() Converte cada imagem para um array
          labels.append(category_idx) # Atribui uma classe numérica

  data = np.array(data) # Converte a lista em um array
  labels = np.array(labels) # Converte a lista em um array

  print(f'There were found {len(data)} images and {len(categories)} labels.')

  return data, labels

In [ ]:
height, width = 128, 128
resize_dimensions = (height, width)
data, labels = load_image_dataset(data_dir, resize_dimensions)

There were found 342 images and 4 labels.


## **Data Processing**

The function `train_test_split()` from sci-kit learn is well largely used to split a dataset between trainning and test (or validation). The argument `shuffle=True` guarantees the data will be shuffled prior to division, thus minimizing any bias. The argument `stratify=labels` states that examples from both classes will exist in derived dataset.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data, labels,
                                                    test_size=0.2, # Test dataset = 20% of data
                                                    shuffle=True, # Prevents bias
                                                    stratify=labels)

## **Training**

The object `classifier` is created as an instance from class `SVC()`. Initially, the `classifier` is created using default arguments.

The variable `parameters` contains two keys. The first value of the pair (key, value) possesses three possible values for the `gamma` argument, while the second value holds four possible values for the argument `C`. Both are entries for `SVC()`. Then, twelve `classifiers` (3 `gammas` X 4 `C`) are generated.

Class `GridSearchCV()` receives the forementioned arguments and creates the twelve classifiers. The method `.fit()` performs the training of those 12 `classifiers`. Finally, the atribute `best_estimator_` finds the best `classifier`. The models are _Support Vector Machine_ `SVM` ML models.

In [ ]:
classifier = SVC()

In [ ]:
parameters = [{'gamma': [0.01, 0.001, 0.0001], 'C': [1, 10, 100, 1000]}]

In [ ]:
grid_search = GridSearchCV(classifier, parameters)

In [ ]:
grid_search.fit(x_train, y_train)

GridSearchCV(estimator=SVC(),
             param_grid=[{'C': [1, 10, 100, 1000],
                          'gamma': [0.01, 0.001, 0.0001]}])

## **Performance**

In [ ]:
best_estimator = grid_search.best_estimator_

In [ ]:
y_pred = best_estimator.predict(x_test)

In [ ]:
score = accuracy_score(y_pred, y_test)

In [ ]:
print('Score: {}%'.format(round(score * 100)))

Score: 99%


In [ ]:
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         7
           1       0.89      1.00      0.94         8
           2       1.00      1.00      1.00         6
           3       1.00      0.98      0.99        48

    accuracy                           0.99        69
   macro avg       0.97      0.99      0.98        69
weighted avg       0.99      0.99      0.99        69



In the following cells an image file is read and tested using the best_estimator `classifier`. The input shall mirror the inputs used to train the model, thus a few operations are required. The result is an array containing the image pixels values divided by 255 and with one single chanel.

This data is not readable by te model, so it is necessary to `reshape` the data into an array of arrays by applying the method `reshape(1, -1)`.

In [ ]:
img_path = os.path.join('/content/gdrive/MyDrive/CCI/TCC/BaseImagensTecido/FS_1/LH/LH_imgt02_hflip.bmp')
img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = cv2.resize(img, (128, 128))
img = img / 255
img = img.flatten()

In [ ]:
prediction = best_estimator.predict(img.reshape(1, -1))
print(prediction[0])

2


In [ ]:
img

array([0.18039216, 0.17647059, 0.19215686, ..., 0.20392157, 0.20392157,
       0.22352941])

In [ ]:
img.reshape(1, -1)

array([[0.18039216, 0.17647059, 0.19215686, ..., 0.20392157, 0.20392157,
        0.22352941]])

## **Save the model**

In [ ]:
pickle.dump(best_estimator, open('./model.p', 'wb'))